In [1]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
import pandas as pd
import numpy as np

%load_ext rpy2.ipython

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


In [2]:
%%R -o df_swimming
library(SwimmeR)
# Scrape the 2024 Olympic 100m Free Final
results <- read_results("https://www.omegatiming.com/File/00011800030101EE0101FFFFFFFFFF01.pdf")
df_swimming <- swim_parse(results, splits = TRUE)

Joining with `by = join_by(Row_Numb)`
Joining with `by = join_by(Row_Numb)`


Columns 'Prelims_Time' and 'Finals_Time' have been changed to 'Prelims' and 'Finals' respectively.  Please update your workflows.


In [3]:
df_swimming

,Place,Heat,Lane,Name,Team,Reaction_Time,Finals,DQ,Exhibition,Event,Split_50,Split_100,Split_150,Split_200
1,1,7,4,HOBSON Luke,LAKE,0.78,1:45.95,0,0,AM Men's 200m Freestyle Heats,24.69,26.75,27.13,27.38
2,2,5,2,PIERONI Blake,ISC,0.65,1:46.09,0,0,AM Men's 200m Freestyle Heats,24.95,26.78,26.94,27.42
3,3,7,5,KIBLER Drew,NYAC,0.61,1:46.39,0,0,AM Men's 200m Freestyle Heats,24.61,26.90,27.34,27.54
4,4,7,3,CURRY Brooks,CAL,0.65,1:46.49,0,0,AM Men's 200m Freestyle Heats,24.46,27.02,27.74,27.27
5,5,5,6,SAMMON Patrick,SUN,0.63,1:46.72,0,0,AM Men's 200m Freestyle Heats,24.82,26.98,27.39,27.53
6,6,6,4,SMITH Kieran,RAC,0.68,1:46.77,0,0,AM Men's 200m Freestyle Heats,24.85,27.16,27.42,27.34
7,7,5,3,CARROZZA Coby,TXLA,0.75,1:46.81,0,0,AM Men's 200m Freestyle Heats,25.06,26.92,27.39,27.44
8,8,3,8,DIEHL Daniel,WOLF,0.66,1:46.83,0,0,AM Men's 200m Freestyle Heats,25.09,26.77,27.49,27.48
9,9,6,5,MITCHELL Jake,FLOR,0.70,1:47.01,0,0,AM Men's 200m Freestyle Heats,24.82,27.09,27.61,27.49
10,10,4,6,GUILIANO Chris,ND,0.62,1:47.05,0,0,AM Men's 200m Freestyle Heats,23.94,27.22,27.84,28.05


In [9]:
olympic_df

,Name,Age,Team,Prelims,Finals,Points,DQ,Exhibition,Event,Split_50,...,Split_750,Split_800,Split_850,Split_900,Split_950,Split_1000,Split_1050,Split_1100,Split_1150,Split_1200
1,50m Freestyle,None,None,None,None,NaN,0.0,0.0,None,None,...,None,None,None,None,None,None,None,None,None,None
2,200m Freestyle,None,None,None,1:53.27,NaN,1.0,0.0,None,None,...,None,None,None,None,None,None,None,None,None,None
3,800m Freestyle,None,None,None,None,NaN,0.0,0.0,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1500m Freestyle,None,None,None,None,NaN,0.0,0.0,None,None,...,None,None,None,None,None,None,None,None,None,None
5,100m Backstroke,None,None,None,None,NaN,0.0,0.0,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,200m Breaststroke,23,4,None,None,NaN,0.0,0.0,Event Event,None,...,None,None,None,None,None,None,None,None,None,None
299,100m Butterfly,32,9,None,None,NaN,0.0,0.0,Event Event,None,...,None,None,None,None,None,None,None,None,None,None
300,200m Butterfly,19,5,None,None,NaN,0.0,0.0,Event Event,None,...,None,None,None,None,None,None,None,None,None,None
301,200m Medley,34,7,None,None,NaN,0.0,0.0,Event Event,None,...,None,None,None,None,None,None,None,None,None,None


In [23]:
%%R -o events -o phases -o results -o splits -o relay_members -o careers -o fina_join

file <- "https://github.com/adghayes/swimset/raw/master/data/fina.rda"
load(url(file)) # Use the 'raw' GitHub link
print(obj_name)

[1] "events"        "phases"        "results"       "splits"       
[5] "relay_members" "careers"       "fina_join"    


In [25]:
for name, df in {
    "events": events,
    "phases": phases,
    "results": results,
    "splits": splits,
    "relay_members": relay_members,
    "careers": careers,
}.items():
    print(f"\n{name}")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))



events
Shape: (10313, 14)
Columns: [np.str_('competition'), np.str_('gender'), np.str_('distance'), np.str_('style'), np.str_('relay'), np.str_('year'), np.str_('location_name'), np.str_('start_day'), np.str_('end_day'), np.str_('start_month'), np.str_('end_month'), np.str_('event_id'), np.str_('series'), np.str_('pool')]

phases
Shape: (39121, 10)
Columns: [np.str_('event_id'), np.str_('phase_id'), np.str_('phase_label'), np.str_('phase_html_id'), np.str_('phase_day'), np.str_('phase_month'), np.str_('phase_size'), np.str_('phase_type'), np.str_('dup'), np.str_('is_summary')]

results
Shape: (281310, 14)
Columns: [np.str_('result_id'), np.str_('phase_id'), np.str_('rank'), np.str_('heat_rank'), np.str_('ioc_code'), np.str_('time'), np.str_('time_behind'), np.str_('record_type'), np.str_('family_name'), np.str_('first_name'), np.str_('rt'), np.str_('points'), np.str_('status'), np.str_('str_time')]

splits
Shape: (771002, 4)
Columns: [np.str_('result_id'), np.str_('split'), np.str_('s

In [26]:
# 1. Merge phases with events
df = phases.merge(events, on="event_id", how="left")

# 2. Merge results
df = results.merge(df, on="phase_id", how="left")

# 3. Optional: add career info (join by name)
df = df.merge(
    careers,
    on=["first_name", "family_name"],
    how="left"
)

df

,result_id,phase_id,rank,heat_rank,ioc_code,time,time_behind,record_type,family_name,first_name,...,series,pool,n,first_year,last_year,year_range,longest_gap,end_gap,start_gap,countries
0,1,1,1,1,CHN,99.10,None,None,None,None,...,World Cup,25m,1,1986,1986,0,0.0,0.0,0.0,1
1,2,1,2,2,AUS,100.08,00.98,None,None,None,...,World Cup,25m,1,1986,1986,0,0.0,0.0,0.0,1
2,3,1,3,3,NED,100.99,01.89,None,None,None,...,World Cup,25m,1,1986,1986,0,0.0,0.0,0.0,1
3,4,1,4,4,GER,103.98,04.88,None,None,None,...,World Cup,25m,1,1986,1986,0,0.0,0.0,0.0,1
4,5,1,5,5,HKG,104.54,05.44,None,None,None,...,World Cup,25m,1,1986,1986,0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281305,447218,39121,41,41,ZIM,133.44,14.42,None,FOGARTY,Catherine Helen,...,Marathon Series,50m,3,2018,2018,0,0.0,2018.0,2018.0,1
281306,447219,39121,42,42,HKG,133.61,14.59,None,HUNG,Sze Ki Celeste,...,Marathon Series,50m,8,1991,2018,27,27.0,2018.0,1991.0,1
281307,447220,39121,43,43,FIJ,136.79,17.77,None,BIRCH,Angela,...,Marathon Series,50m,5,2018,2018,0,0.0,2018.0,2018.0,1
281308,447221,39121,44,44,FIJ,138.45,19.43,None,MUNCH,Cina,...,Marathon Series,50m,4,2018,2018,0,0.0,2018.0,2018.0,1


In [27]:
df.columns

Index(['result_id', 'phase_id', 'rank', 'heat_rank', 'ioc_code', 'time',
       'time_behind', 'record_type', 'family_name', 'first_name', 'rt',
       'points', 'status', 'str_time', 'event_id', 'phase_label',
       'phase_html_id', 'phase_day', 'phase_month', 'phase_size', 'phase_type',
       'dup', 'is_summary', 'competition', 'gender', 'distance', 'style',
       'relay', 'year', 'location_name', 'start_day', 'end_day', 'start_month',
       'end_month', 'series', 'pool', 'n', 'first_year', 'last_year',
       'year_range', 'longest_gap', 'end_gap', 'start_gap', 'countries'],
      dtype='object')

In [30]:
df_with_splits = df.merge(splits, on="result_id", how="left")
df_with_splits = df_with_splits[['rank', 'heat_rank', 'ioc_code', 'time', 'time_behind', 'family_name', 'first_name', 'rt', 'phase_size', 'gender', 'style', 'year']]

In [33]:
final_df = df_with_splits[df_with_splits['year'] > 2000]

In [35]:
final_df['year'].min()

np.int32(2001)

In [36]:
final_df

,rank,heat_rank,ioc_code,time,time_behind,family_name,first_name,rt,phase_size,gender,style,year
0,1,1,CHN,99.10,None,None,None,None,10,Mixed,Medley,2017
1,1,1,CHN,99.10,None,None,None,None,10,Mixed,Medley,2017
2,1,1,CHN,99.10,None,None,None,None,10,Mixed,Medley,2017
3,1,1,CHN,99.10,None,None,None,None,10,Mixed,Medley,2017
4,2,2,AUS,100.08,00.98,None,None,None,10,Mixed,Medley,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
868885,44,44,FIJ,138.45,19.43,MUNCH,Cina,None,45,Women,Freestyle,2018
868886,44,44,FIJ,138.45,19.43,MUNCH,Cina,None,45,Women,Freestyle,2018
868887,44,44,FIJ,138.45,19.43,MUNCH,Cina,None,45,Women,Freestyle,2018
868888,44,44,FIJ,138.45,19.43,MUNCH,Cina,None,45,Women,Freestyle,2018
